# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    dirs[:] = [d for d in dirs if d not in ['.ipynb_checkpoints']]
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Number of records in consolidated csv file: ', sum(1 for line in f))

Number of records in consolidated csv file:  6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)    

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### - Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### 1. Query 1: show the artist, song title and song's length with sessionId = 338 and itemInSession = 4

#### 1.1. Create a table for query 1

In [8]:
# Populate CREATE TABLE statment into query variable and execute it.
create_query = "CREATE TABLE IF NOT EXISTS artist_song_length_by_session_item"
create_query = create_query + "(sessionId int, itemInSession int, artist text, song text, length float , PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create_query)
except Exception as e:
    print(e)        

*the reason to use sessionId and itemInSession as primary key is because the query uses these two criteria to filter the table, which in return could be able to create partition as partition key and identify each row as primary key.*

#### 1.2. Read files and insert data into the table for query 1

In [9]:
# Set up the CSV file
file = 'event_datafile_new.csv'

# Open the csv file in read mode and close it once the INSERT statement is finished
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Loop through each line and insert the data into the table for the first query 
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable with variables -- sessionId, itemInSession
        # artist, song, length
        insert_query = "INSERT INTO artist_song_length_by_session_item (sessionId, itemInSession, artist, song, length)"
        insert_query = insert_query + "VALUES(%s, %s, %s, %s, %s)"

        # Assign corresponding columns to columns in the INSERT statement
        session.execute(insert_query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### 1.3. Do a SELECT to verify that the data have been inserted into each table

In [10]:
# SELECT statement to get artist, song, length based on given sessionId and itemInSession
query = "SELECT artist, song, length FROM artist_song_length_by_session_item WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Loop through the SELECT query results and display the output
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Query 2: show name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### 2.1. Create a table for query 2

In [11]:
# Populate CREATE TABLE statment into query variable and execute it.
create_query = "CREATE TABLE IF NOT EXISTS artist_song_by_user_session_item"
create_query = create_query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession));"
try:
    session.execute(create_query)
except Exception as e:
    print(e)

*the reason to use userId, sessionId and itemInSession as primary key is because the query uses userId and sessionId to filter the table and orders the filtered result by itemInSession, which in return means the userId could become a partition key together with sessionId as primary key. Moreover, to use itemInSession in the ORDER BY statement, itemInSession should show up in the primary key in order.*

#### 2.2. Read files and insert data into the table for query 2

In [12]:
# Set up the CSV file
file = 'event_datafile_new.csv'

# Open the csv file in read mode and close it once the INSERT statement is finished
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Loop through each line and insert the data into the table for the first query 
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable with variables -- userId, sessionId
        # itemInSession, artist, song, firstName, lastName
        insert_query = "INSERT INTO artist_song_by_user_session_item (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        insert_query = insert_query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        # Assign corresponding columns to columns in the INSERT statement
        session.execute(insert_query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### 2.3. Do a SELECT to verify that the data have been inserted into the table

In [13]:
# SELECT statement to get artist, song, first and last name of the artist based on given userId and sessionId 
# and order by itemInSession
query = "SELECT artist, song, firstName, lastName FROM artist_song_by_user_session_item WHERE userId=10 AND sessionId=182 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Loop through the SELECT query results and display the output
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Query 3: show every user name (first and last) who listened to the song 'All Hands Against His Own'

#### 3.1. Create a table for query 3

In [14]:
# Populate CREATE TABLE statment into query variable and execute it.
create_query = "CREATE TABLE IF NOT EXISTS user_by_song_userId"
create_query = create_query + "(song text, userId int, firstName text, lastName text , PRIMARY KEY (song, userId))"
try:
    session.execute(create_query)
except Exception as e:
    print(e)

*the reason to use song and userId as primary key is because the query tries to explore the relationship between the users and songs, and userId is unique to each user, which means song and userId could uniquely identify each record.*

#### 3.2. Read files and insert data into the table for query 3

In [15]:
# Set up the CSV file
file = 'event_datafile_new.csv'

# Open the csv file in read mode and close it once the INSERT statement is finished
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Loop through each line and insert the data into the table for the first query 
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable with variables 
        # -- song, first and last name of the artist
        insert_query = "INSERT INTO user_by_song_userId (song, userId, firstName, lastName)"
        insert_query = insert_query + "VALUES(%s, %s, %s, %s)"
        
        # Assign corresponding columns to columns in the INSERT statement
        session.execute(insert_query, (line[9], int(line[10]), line[1], line[4]))

#### 3.3. Do a SELECT to verify that the data have been inserted into the table

In [16]:
# SELECT statement to get the first and last name of the artist based on a given song
query = "select firstName, lastName from user_by_song_userId WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Loop through the SELECT query results and display the output
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
for table in ['artist_song_length_by_session_item', 'artist_song_by_user_session_item', 'user_by_song_userId']:
    query = "DROP TABLE IF EXISTS " + table
    try:
        session.execute(query)
        print(table, 'is dropped')
    except Exception as e:
        print(e)

artist_song_length_by_session_item is dropped
artist_song_by_user_session_item is dropped
user_by_song_userId is dropped


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()